In [ ]:
import importlib
from pathlib import Path

for pkg in [
    "odc.emit",
    "odc.geo",
    "odc.stac",
    "datacube",
    "pystac",
    "dask",
    "distributed",
    "sqlalchemy",
]:
    m = importlib.import_module(pkg)
    pth = str(Path(m.__file__).parent).replace(str(Path.home()), "~")
    v = getattr(m, "__version__", "??")
    pkg = f"{pkg}=={v}"
    print(f"{pkg:35}  # {pth}")

In [ ]:
import os
import sys

for e in os.environ:
    if e.startswith("DB_"):
        os.environ.pop(e, None)

os.environ["ODC_EMIT_DB_URL"] = "postgresql:///datacube?host=/tmp"

In [ ]:
from IPython.display import JSON, display

In [ ]:
import datacube
import odc.emit
import odc.geo
import odc.stac
import sqlalchemy

In [ ]:
from datacube import Datacube
from datacube.cfg.api import ODCConfig, ODCEnvironment
from datacube.storage import BandInfo, measurement_paths
from datacube.testutils.io import eo3_geobox
from datacube.utils.serialise import jsonify_document
from odc.geo.geom import unary_union


def band_to_json(bb):
    return jsonify_document({k: getattr(bb, k) for k in bb.__slots__})


dc = Datacube(env="emit")

In [ ]:
dc.list_products(with_pandas=False)

In [ ]:
p = dc.index.products.get_by_name("emit_l2a")

display(JSON(p.definition))
print("-" * 40)
display(JSON(p.extra_dimensions.dims["wavelength"]))
print("-" * 40)
display(p.load_hints())

In [ ]:
list(dc.index.datasets.count_by_product())

In [ ]:
dss = dc.find_datasets(
    product="emit_l2a",
    lon=(146, 149),
    lat=(-37, -36),
)
display(len(dss), unary_union([ds.extent.exterior for ds in dss]))

In [ ]:
dss = dc.find_datasets(
    product="emit_l2a",
    lon=(146, 149),
    lat=(-37, -36),
    time="2023-10",
)
display(len(dss), unary_union([ds.extent.exterior for ds in dss]))

In [ ]:
ds = dss[0]
gbox = eo3_geobox(ds, "elev")
display(
    ds.time,
    ds.crs,
    ds.extent,
    gbox,
    JSON(ds.product.extra_dimensions.dims["wavelength"]),
)

In [ ]:
bands = {band: BandInfo(ds, band) for band in ds.product.measurements}
display(JSON({name: band_to_json(bb) for name, bb in bands.items()}))

In [ ]:
JSON(ds.metadata_doc)

In [ ]:
!~/.envs/kk/bin/datacube -E emit system check

In [ ]:
#!~/.envs/kk/bin/datacube -E emit product update --allow-unsafe emit.yaml